In [1]:
import torch
import numpy as np

import os
os.chdir('..')
from torch_geometric.loader import DataLoader
from gnn_library.util import train, save, load
from evaluate import evaluate_model, pp_output
import instance_generator as ig
import torch_converter as tc
import evaluate as ev
import osmnx as ox
from util import Dataset

%load_ext autoreload
%autoreload 2

In [2]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

PyTorch has version 1.12.0+cu102
Using device: cuda:2


In [3]:
def define_model(trial):
    args = {
        'processor':         'GENConv',
        'head':              'regression',
        'num_layers':        trial.suggest_int("num_layers{}", 1, 6),
        'num_mlp_layers':    trial.suggest_int("num_mlp_layers{}", 1, 5), # TODO set to larger
        'aggr':              'max',
        'batch_size':        2**trial.suggest_int("log_batch_size", 1, 6), 
        'node_feature_dim':  5,
        'edge_feature_dim':  1,
        'graph_feature_dim': 2,
        'hidden_dim':        2**trial.suggest_int("hidden_dim", 1, 7), # TODO set to 128
        'output_dim':        1,
        'dropout':           trial.suggest_float("dropout", 0, 0.5),
        'epochs':            2**trial.suggest_int("epochs", 2, 8), # TODO set to larger
        'opt':               trial.suggest_categorical("optimizer", ["adam", "adagrad"]),
        'opt_scheduler':     'none',
        'opt_restart':       0,
        'weight_decay':      5e-3, # TODO possibly modify
        'lr':                trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        'device':            device,
        'noise':             0
    }
    return args

In [4]:
def objective(trial):
	args = define_model(trial)
	train_num = 200; test_num = 100

	er_config = {
	'graph_type': 'ER',
	'p': 0.75,
	'weighted': True
	}
	ba_config = {
	'graph_type': 'BA',
	'ba_param': 4,
	'weighted': True
	}
	geom_config = {
		'graph_type': 'GEOM',
		'q': 0.15,
		'd': 2,
		'weighted': True
	}

	rng = np.random.default_rng()


	train_instances = [
		*ig.sample_instances(10, 6, train_num, rng, args, **er_config),
		*ig.sample_instances(10, 6, train_num, rng, args, **ba_config),
		*ig.sample_instances(10, 6, train_num, rng, args, **geom_config),
	]

	test_instances = [
		*ig.sample_instances(10, 6, test_num, rng, args, **er_config),
		*ig.sample_instances(10, 6, test_num, rng, args, **ba_config),
		*ig.sample_instances(10, 6, test_num, rng, args, **geom_config),
	]


	train_data = Dataset(tc._instances_to_train_samples(train_instances, args['head']))
	test_data = Dataset(tc._instances_to_train_samples(test_instances, args['head']))

	train_loader = DataLoader(
	train_data,
	batch_size=args['batch_size'],
	shuffle=True,
	num_workers=4
	)

	test_loader = DataLoader(
	test_data,
	batch_size=args['batch_size'],
	shuffle=True,
	num_workers=4
	)

	### Training

	_, _, test_accuracies, _, _ = train(train_loader, test_loader, args, trial)
	model_accuracy = test_accuracies[-1]
	
	return model_accuracy

In [5]:
import optuna
from optuna.trial import TrialState

study = optuna.create_study(study_name='hyperparam-study', direction='maximize', storage='sqlite:///hyperparam.db', load_if_exists=True)
study.optimize(objective, n_trials=1000, timeout=60000)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
	print("    {}: {}".format(key, value))


[I 2024-01-25 12:13:32,352] Using an existing study with name 'hyperparam-study' instead of creating a new one.
Training:   0%|          | 0/128 [00:00<?, ?Epochs/s]/home/alexhay/.local/lib/python3.10/site-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "
Training: 100%|██████████| 128/128 [19:25<00:00,  9.10s/Epochs]
[I 2024-01-25 12:33:12,285] Trial 5 finished with value: 0.19778189063072205 and parameters: {'num_layers{}': 4, 'num_mlp_layers{}': 4, 'log_batch_size': 4, 'hidden_dim': 3, 'dropout': 0.4558675989687025, 'epochs': 7, 'optimizer': 'adagrad', 'lr': 3.2632239763514146e-05}. Best is trial 5 with value: 0.19778189063072205.
Training: 100%|██████████| 8/8 [02:05<00:00, 15.63s/Epochs]
[I 2024-01-25 12:35:30,410] Trial 6 finished with value: 0.6293958425521851 and parameters: {'num_layers{}': 3, 'num_

Study statistics: 
  Number of finished trials:  81
  Number of pruned trials:  42
  Number of complete trials:  34
Best trial:
  Value:  0.8943089842796326
  Params: 
    num_layers{}: 3
    num_mlp_layers{}: 3
    log_batch_size: 3
    hidden_dim: 5
    dropout: 0.03064464576377201
    epochs: 6
    optimizer: adagrad
    lr: 0.012190706489490367
